In [1]:
import IO_map
import random
import re
import generate_verilog as gv

"""
Wire can be in many groups: A wire can be on many linear paths
    - Every wire is initally in a group with it's parent and child modules
Wire can be in many loops: A wire can be on many looped paths
Groups contains {group_id : [modules in that group]}
    - Initially one group for every input and output pair 
    - Group id is initially the wire id, since each wire is in it's own group
Loops contains {loop_id : [modules in that loop]}

"""
class Fuzz_Run:
    def __init__(self, flattened_lib_path):
        random.seed(0)
        self.flattened_lib_path = flattened_lib_path
        io_map = IO_map.create_IO_map(flattened_lib_path)

        self.all_wires = io_map['bit_input']
        self.all_wires.update(io_map['bit_output']) # Single source of truth
        
        self.mod_IO = io_map['mod_IO']
        self.all_modules = io_map['mod_list']

        self.external_inputs = list(io_map['bit_input'].keys())
        self.external_outputs = list(io_map['bit_output'].keys()) # Unchanged will always output all outputs

        self.module_tree = {}
        for i in range(len(io_map['mod_list'])):
            module = io_map['mod_list'][i]
            self.module_tree[module] = {"depth" : 0, "tree":i, "children" : [], "parents":[]}
    
    def linear_rewire(self):
        """
        Rule: 
        1. The output of a wire cannot connect to it's predecessor in the same tree. Can pick from one of its successors, or another tree.
        """
        # We use external_outputs because all module outputs will be a external output
        potential_outputs = self.external_outputs.copy()
        random.shuffle(potential_outputs)

        for output_wire_id in potential_outputs:
            output_wire = self.all_wires[output_wire_id]
            output_wire_module = output_wire.input #The module that the wire is attatched to
            output_tree = self.module_tree[output_wire_module]["tree"]
            output_depth = self.module_tree[output_wire_module]["depth"]

            # Pick a module that is not from the tree, or at a higher depth than current. Sample without replacement.
            potential_inputs = self.all_modules.copy()
            random.shuffle(potential_inputs)
            chosen_module = None
            chosen_module_tree = None
            chosen_module_depth = None

            for input_mod in potential_inputs:
                input_tree = module_tree[input_mod]["tree"]
                input_depth = module_tree[input_mod]["depth"]
                if input_tree != output_tree or input_depth >= output_depth:
                    chosen_module = input_mod
                    chosen_module_tree = input_tree
                    chosen_module_depth = input_depth
                    break

            if chosen_module == None:
                print("No available module to form linear connection")
                break

            # Pick a random input to chosen_module
            chosen_wire_id = random.sample(self.mod_IO[chosen_module]['input'], 1)
            chosen_wire = self.all_wires[chosen_wire_id]

            # Add new wire connection 
            output_wire.output.append(chosen_wire.output)

            # Disconnect the wire's output
            chosen_wire.output.remove(chosen_input_wire_port)
            # If this wire recieves its data from ext input, then we need to remove it
            # if its not outputting to any ports. Otherwise its receiving it's data from
            # a module, and will always be connected to the ext output.
            if chosen_input_wire.type == IO_map.Wire.Wire_Type.ITM and len(chosen_input_wire.output) == 0:
                self.all_wires.pop(chosen_input_wire.id)
                self.external_inputs.remove(chosen_input_wire.id)

            # Modify tree
            if (chosen_module_tree != output_tree):
                self.module_tree[chosen_module]['tree'] = self.module_tree[output_tree]['tree'] # Merge trees

            if(chosen_module_depth == output_depth):
                self.module_tree[chosen_module]['depth'] = self.module_tree[output_tree]['depth'] + 1 # Adjust depth

            # Add to wire to internal_connections

            # If this wire recieves its data from ext input, then we need to remove it
            # if its not outputting to any ports. Otherwise its receiving it's data from
            # a module, and will always be connected to the ext output.
            if chosen_input_wire.type == IO_map.Wire.Wire_Type.ITM and len(chosen_input_wire.output) == 0:
                self.all_wires.pop(chosen_input_wire.id)
                self.external_inputs.remove(chosen_input_wire.id)
            
            print("Found linear rewiring combination and successfully rewired")

In [16]:
test_fuzz = Fuzz_Run("/module-fuzz/test_library/flattened_IO")
test_fuzz.linear_rewire()

KeyError: 'multiplier_wrapper'